# Unit 3

## Error Handling and Validation

Welcome back\! 🚀 In the last two lessons, you learned how to set up a **FastAPI backend** for a **Code Review Assistant** and how to manage changesets using API endpoints. Now, we will focus on making your API more reliable and secure by handling errors and validating user input.

Error handling and validation are essential for any web application. They help you:

  * **Prevent bad data from entering your system.**
  * **Give clear feedback to users when something goes wrong.**
  * **Protect your application from crashes and security issues.**

In this lesson, you'll learn how to use **FastAPI** and **Pydantic** to validate incoming data and handle different types of errors gracefully. This will make your API safer and easier to use.

-----

## Quick Recall: FastAPI Endpoints and Pydantic Models

Before we dive in, let's quickly remind ourselves how FastAPI endpoints and Pydantic models work together.

In FastAPI, you define endpoints using Python functions. When you want to receive structured data (like a changeset submission), you use a **Pydantic model** to describe what the data should look like.

For example, in a previous lesson, you might have seen something like this:

```python
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class ChangesetRequest(BaseModel):
    title: str
    author: str

@app.post("/api/changesets")
def submit_changeset(request: ChangesetRequest):
    return {"message": "Received", "title": request.title}
```

Here, `ChangesetRequest` is a Pydantic model that describes the expected fields. FastAPI automatically checks that incoming requests match this model and converts the data for you.

-----

## Validating Input with Pydantic

Now, let's see how to add more detailed validation to our input data using **Pydantic**. This helps catch mistakes early and gives users helpful error messages.

Suppose we want to make sure that:

  * **The file path is not empty and doesn't allow directory traversal (like `../`).**
  * **The diff content is not empty and not too large.**
  * **The author field is a valid username or email.**

We can use Pydantic's **`@field_validator`** to add these checks.

Let's start by defining a model for a file change:

```python
from pydantic import BaseModel, field_validator

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str

    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        return v.strip()

    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        return v
```

**Explanation:**

  * The **`@field_validator`** decorator lets you write custom checks for each field.
  * For **`file_path`**, we check that it is not empty and does not contain `..` or start with `/`, which could be used to access files outside the allowed directory.
  * For **`diff_content`**, we check that it is not empty and not too large.

If a user sends invalid data, FastAPI will automatically return a helpful error message.

-----

## Handling Errors in FastAPI

Even with good validation, things can still go wrong. For example, the database might reject a request, or your code might raise a custom error. FastAPI lets you handle these situations using **exception handlers**.

Let's look at how to handle three common types of errors: **Custom application errors**, **Validation errors**, and **Database errors**.

### 1\. Custom Application Errors

You can define your own exception class for special situations:

```python
class CodeReviewException(Exception):
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)
```

Then, you can tell FastAPI how to handle this error:

```python
from fastapi import Request
from fastapi.responses import JSONResponse

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )
```

Now, if you raise `CodeReviewException("Something went wrong", 400)`, the user will get a clear error message.

### 2\. Validation Errors

FastAPI already handles validation errors for you, but you can customize the response:

```python
from fastapi.exceptions import RequestValidationError
from fastapi import status

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": exc.errors()}
    )
```

This sends a clear message when the input data does not match your Pydantic model.

### 3\. Database Errors

If you use a database, you might get errors like unique constraint violations. You can handle these too:

```python
from sqlalchemy.exc import IntegrityError

@app.exception_handler(IntegrityError)
async def database_exception_handler(request: Request, exc: IntegrityError):
    return JSONResponse(
        status_code=status.HTTP_409_CONFLICT,
        content={"error": "Database constraint violation"}
    )
```

This way, your API always returns a clear and consistent error message.

-----

## Bringing It Together: Example Endpoint with Validation and Error Handling

Let's see how all these pieces fit together in a real endpoint. Here's how you might write an endpoint to submit a changeset, with validation and error handling:

```python
from fastapi import BackgroundTasks, Depends

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: DummyDBSession = Depends(get_session)
):
    try:
        # Create changeset
        changeset = Changeset(
            title=request.title,
            description=request.description,
            author=request.author
        )
        db.add(changeset)
        db.flush()

        # Add files
        for file_req in request.files:
            changeset_file = ChangesetFile(
                changeset_id=changeset.id,
                file_path=file_req.file_path,
                diff_content=file_req.diff_content
            )
            db.add(changeset_file)

        db.commit()
        
        # Process review in background
        background_tasks.add_task(process_changeset_review, changeset.id)
        
        return {"message": "Changeset submitted", "changeset_id": changeset.id}
    except Exception as e:
        db.rollback()
        raise CodeReviewException("Failed to create changeset", 500)
```

**Explanation:**

  * The endpoint receives a **`ChangesetRequest`** (which uses Pydantic validation).
  * If anything goes wrong (for example, a database error), we roll back the transaction and raise a **`CodeReviewException`**.
  * The exception handlers we set up earlier will catch this and return a clear error message to the user.

### Example Output:

If the request is valid, the response will be:

```json
{
  "message": "Changeset submitted",
  "changeset_id": 1
}
```

If the input is invalid (for example, the file path is empty), the response will be:

```json
{
  "error": "Invalid input data",
  "details": [
    {
      "loc": ["body", "files", 0, "file_path"],
      "msg": "File path cannot be empty",
      "type": "value_error"
    }
  ]
}
```

If there is a database error, the response will be:

```json
{
  "error": "Database constraint violation"
}
```

-----

## Summary and What's Next

In this lesson, you learned how to:

  * Use **Pydantic** to validate incoming data and provide helpful error messages.
  * Handle different types of errors in FastAPI using **custom exception handlers**.
  * Combine validation and error handling in a real API endpoint.

These skills will help you build APIs that are **robust, secure, and user-friendly**. Up next, you'll get to practice these concepts with hands-on exercises. Good luck, and keep up the great work\! ✨

## Add Email Validation with Pydantic

Nice work learning about Pydantic field validators! Now it's time to put that knowledge into practice by adding email validation to a user registration system.

You have a UserRequest model that accepts user information, but currently, the author field doesn't check whether the email format is valid. Your objective is to add a field validator that ensures users provide a proper email address.

Here's what you need to do:

Add a @field_validator('author') method to the UserRequest class
Use regex pattern matching to check whether the email format is valid
Raise a ValueError with the message "Invalid email format" if the email doesn't match the expected pattern
Return the cleaned email (stripped of whitespace) if it's valid
The regex pattern you should use is: r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

Once you complete this validation, you'll have hands-on experience with one of the most common validation tasks in web development!

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

class UserRequest(BaseModel):
    name: str
    author: str
    
    # TODO: Add a field validator for the 'author' field that validates email format

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": exc.errors()}
    )

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        
        # Basic security check - no parent directory access
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        
        return v.strip()
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Title cannot be empty')
        
        if len(v) > 200:
            raise ValueError('Title too long (max 200 characters)')
        
        return v.strip()
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Author cannot be empty')
        
        # Basic email or username validation
        if not re.match(r'^[a-zA-Z0-9._-]+(@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})?$', v):
            raise ValueError('Invalid author format')
        
        return v.strip()
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v):
        if not v or len(v) == 0:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:  # Limit to 20 files per changeset
            raise ValueError('Too many files (max 20)')
        
        return v

@app.post("/api/users", response_model=dict)
def create_user(request: UserRequest):
    """Create a new user with validation"""
    return {"message": "User created successfully", "name": request.name, "email": request.author}

@app.post("/api/changesets", response_model=dict)
def submit_changeset(request: ChangesetRequest, background_tasks: BackgroundTasks):
    """Submit a new changeset for review with validation"""
    try:
        # Simulate changeset creation without database
        changeset_id = 1
        logger.info(f"Changeset {changeset_id} created by {request.author}")
        
        return {"message": "Changeset submitted", "changeset_id": changeset_id}
        
    except Exception as e:
        logger.error(f"Failed to create changeset: {str(e)}")
        raise CodeReviewException("Failed to create changeset", 500)

# Test the validation
if __name__ == "__main__":
    # Test valid email
    try:
        user = UserRequest(name="John Doe", author="john.doe@example.com")
        print(f"✓ Valid email: {user.author}")
    except Exception as e:
        print(f"✗ Error: {e}")
    
    # Test invalid email
    try:
        user = UserRequest(name="Jane Doe", author="invalid-email")
        print(f"✓ Should not reach here: {user.author}")
    except Exception as e:
        print(f"✓ Correctly caught invalid email: {e}")
    
    print("Email validation test completed!")
```

You got it\! I'll add the email validation logic to the `UserRequest` model using the specified regex pattern.

Here is the updated code:

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

class UserRequest(BaseModel):
    name: str
    author: str
    
    # Implementation of the email validation logic
    @field_validator('author')
    @classmethod
    def validate_email_format(cls, v: str):
        # Email regex pattern provided in the prompt
        email_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        
        cleaned_email = v.strip()
        
        if not re.match(email_regex, cleaned_email):
            raise ValueError('Invalid email format')
            
        # Return the cleaned email if validation passes
        return cleaned_email

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": exc.errors()}
    )

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        
        # Basic security check - no parent directory access
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        
        return v.strip()
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Title cannot be empty')
        
        if len(v) > 200:
            raise ValueError('Title too long (max 200 characters)')
        
        return v.strip()
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Author cannot be empty')
        
        # Basic email or username validation
        if not re.match(r'^[a-zA-Z0-9._-]+(@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})?$', v):
            raise ValueError('Invalid author format')
        
        return v.strip()
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v):
        if not v or len(v) == 0:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:  # Limit to 20 files per changeset
            raise ValueError('Too many files (max 20)')
        
        return v

@app.post("/api/users", response_model=dict)
def create_user(request: UserRequest):
    """Create a new user with validation"""
    return {"message": "User created successfully", "name": request.name, "email": request.author}

@app.post("/api/changesets", response_model=dict)
def submit_changeset(request: ChangesetRequest, background_tasks: BackgroundTasks):
    """Submit a new changeset for review with validation"""
    try:
        # Simulate changeset creation without database
        changeset_id = 1
        logger.info(f"Changeset {changeset_id} created by {request.author}")
        
        return {"message": "Changeset submitted", "changeset_id": changeset_id}
        
    except Exception as e:
        logger.error(f"Failed to create changeset: {str(e)}")
        raise CodeReviewException("Failed to create changeset", 500)

# Test the validation
if __name__ == "__main__":
    # Test valid email
    try:
        user = UserRequest(name="John Doe", author="john.doe@example.com")
        print(f"✓ Valid email: {user.author}")
    except Exception as e:
        print(f"✗ Error: {e}")
    
    # Test invalid email
    try:
        user = UserRequest(name="Jane Doe", author="invalid-email")
        print(f"✓ Should not reach here: {user.author}")
    except Exception as e:
        print(f"✓ Correctly caught invalid email: {e}")
    
    # Test email with extra whitespace
    try:
        user = UserRequest(name="Test User", author="  test.user@company.net ")
        print(f"✓ Valid email (stripped): '{user.author}'")
        assert user.author == "test.user@company.net"
    except Exception as e:
        print(f"✗ Error: {e}")
    
    print("Email validation test completed!")
```

## Create Custom Exception Class

Excellent work with Pydantic field validators and FastAPI exception handlers! Now you'll take error handling to the next level by creating your own custom exception class.

Currently, the user registration endpoint uses generic Python exceptions like ValueError when user data is invalid. This makes it difficult to handle different types of errors properly and to give users clear feedback. Your task is to create a custom UserValidationException class that will organize your error handling much better.

Here's what you need to do:

Create a UserValidationException class that inherits from Python's base Exception class
Add an __init__ method that accepts a message parameter and an optional status_code parameter (defaulting to 400)
Store both parameters as instance attributes and call the parent constructor
Replace all the generic ValueError exceptions in the user endpoint with your new custom exception
Add an exception handler that returns proper JSON responses for your custom exception
This will give you hands-on experience with one of the most important patterns in professional API development — using domain-specific exceptions for better error organization and user experience!

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks, Depends
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import datetime
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Database models
class Changeset(Base):
    __tablename__ = "changesets"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200))
    description = Column(Text, nullable=True)
    author = Column(String(100))
    status = Column(String(20), default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

class ChangesetFile(Base):
    __tablename__ = "changeset_files"
    id = Column(Integer, primary_key=True, index=True)
    changeset_id = Column(Integer, ForeignKey("changesets.id"))
    file_path = Column(String(255))
    diff_content = Column(Text)

# Create database tables
Base.metadata.create_all(bind=engine)

# Session dependency
def get_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

app = FastAPI()

# TODO: Create the UserValidationException class that inherits from Exception
class UserValidationException(Exception):
    """Custom exception for user validation errors"""
    # TODO: Add __init__ method with message and optional status_code parameters

class UserRequest(BaseModel):
    username: str
    email: str

# TODO: Add exception handler for UserValidationException

@app.post("/api/users", response_model=dict)
def create_user(request: UserRequest):
    """Create a new user with validation"""
    # Validate username
    if not request.username or len(request.username.strip()) == 0:
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Username cannot be empty")
    
    if len(request.username.strip()) < 3:
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Username must be at least 3 characters long")
    
    if len(request.username.strip()) > 50:
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Username cannot be longer than 50 characters")
    
    if not re.match(r'^[a-zA-Z0-9_-]+$', request.username.strip()):
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Username can only contain letters, numbers, underscores, and hyphens")
    
    # Validate email
    if not request.email or len(request.email.strip()) == 0:
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Email cannot be empty")
    
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(email_pattern, request.email.strip()):
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Invalid email format")
    
    return {"message": "User created successfully", "username": request.username, "email": request.email}

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": exc.errors()}
    )

@app.exception_handler(IntegrityError)
async def database_exception_handler(request: Request, exc: IntegrityError):
    logger.error(f"Database error: {str(exc)}")
    return JSONResponse(
        status_code=status.HTTP_409_CONFLICT,
        content={"error": "Database constraint violation"}
    )

def process_changeset_review(changeset_id: int):
    pass

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        
        # Basic security check - no parent directory access
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        
        return v.strip()
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Title cannot be empty')
        
        if len(v) > 200:
            raise ValueError('Title too long (max 200 characters)')
        
        return v.strip()
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Author cannot be empty')
        
        # Basic email or username validation
        if not re.match(r'^[a-zA-Z0-9._-]+(@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})?$', v):
            raise ValueError('Invalid author format')
        
        return v.strip()
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v):
        if not v or len(v) == 0:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:  # Limit to 20 files per changeset
            raise ValueError('Too many files (max 20)')
        
        return v

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_session)
):
    """Submit a new changeset for review with validation"""
    try:
        # Create changeset
        changeset = Changeset(
            title=request.title,
            description=request.description,
            author=request.author
        )
        db.add(changeset)
        db.flush()
        
        # Add files
        for file_req in request.files:
            changeset_file = ChangesetFile(
                changeset_id=changeset.id,
                file_path=file_req.file_path,
                diff_content=file_req.diff_content
            )
            db.add(changeset_file)
        
        db.commit()
        logger.info(f"Changeset {changeset.id} created by {request.author}")
        
        # Process review in background
        background_tasks.add_task(process_changeset_review, changeset.id)
        
        return {"message": "Changeset submitted", "changeset_id": changeset.id}
        
    except Exception as e:
        db.rollback()
        logger.error(f"Failed to create changeset: {str(e)}")
        raise CodeReviewException("Failed to create changeset", 500)

# Test the custom exception functionality
if __name__ == "__main__":
    print("Testing UserValidationException class...")
    
    # Test 1: Creating custom exception
    try:
        exc = UserValidationException("Test message", 400)
        print(f"✓ Custom exception created: {exc.message}, status_code: {exc.status_code}")
    except Exception as e:
        print(f"✗ Failed to create custom exception: {e}")
        exit(1)
    
    # Test 2: Testing validation logic
    test_cases = [
        {"username": "", "email": "test@example.com", "expected_error": "Username cannot be empty"},
        {"username": "ab", "email": "test@example.com", "expected_error": "Username must be at least 3 characters long"},
        {"username": "valid_user", "email": "", "expected_error": "Email cannot be empty"},
        {"username": "valid_user", "email": "invalid_email", "expected_error": "Invalid email format"},
    ]
    
    print("\nTesting validation logic...")
    for i, test_case in enumerate(test_cases, 1):
        try:
            request = UserRequest(username=test_case["username"], email=test_case["email"])
            create_user(request)
            print(f"✗ Test {i} failed: Expected error but validation passed")
        except UserValidationException as e:
            if e.message == test_case["expected_error"]:
                print(f"✓ Test {i} passed: {e.message}")
            else:
                print(f"✗ Test {i} failed: Expected '{test_case['expected_error']}', got '{e.message}'")
        except Exception as e:
            print(f"✗ Test {i} failed: Unexpected error type: {type(e).__name__}")
    
    # Test 3: Valid user creation
    try:
        request = UserRequest(username="valid_user", email="user@example.com")
        result = create_user(request)
        print(f"✓ Valid user creation test passed: {result['message']}")
    except Exception as e:
        print(f"✗ Valid user creation test failed: {e}")
    
    print("\n✓ All tests completed! Custom exception implementation is working correctly.")

```

## Create Custom Exception Class

Excellent work with Pydantic field validators and FastAPI exception handlers! Now you'll take error handling to the next level by creating your own custom exception class.

Currently, the user registration endpoint uses generic Python exceptions like ValueError when user data is invalid. This makes it difficult to handle different types of errors properly and to give users clear feedback. Your task is to create a custom UserValidationException class that will organize your error handling much better.

Here's what you need to do:

Create a UserValidationException class that inherits from Python's base Exception class
Add an __init__ method that accepts a message parameter and an optional status_code parameter (defaulting to 400)
Store both parameters as instance attributes and call the parent constructor
Replace all the generic ValueError exceptions in the user endpoint with your new custom exception
Add an exception handler that returns proper JSON responses for your custom exception
This will give you hands-on experience with one of the most important patterns in professional API development — using domain-specific exceptions for better error organization and user experience!

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks, Depends
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import datetime
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Database models
class Changeset(Base):
    __tablename__ = "changesets"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200))
    description = Column(Text, nullable=True)
    author = Column(String(100))
    status = Column(String(20), default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

class ChangesetFile(Base):
    __tablename__ = "changeset_files"
    id = Column(Integer, primary_key=True, index=True)
    changeset_id = Column(Integer, ForeignKey("changesets.id"))
    file_path = Column(String(255))
    diff_content = Column(Text)

# Create database tables
Base.metadata.create_all(bind=engine)

# Session dependency
def get_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

app = FastAPI()

# TODO: Create the UserValidationException class that inherits from Exception
class UserValidationException(Exception):
    """Custom exception for user validation errors"""
    # TODO: Add __init__ method with message and optional status_code parameters

class UserRequest(BaseModel):
    username: str
    email: str

# TODO: Add exception handler for UserValidationException

@app.post("/api/users", response_model=dict)
def create_user(request: UserRequest):
    """Create a new user with validation"""
    # Validate username
    if not request.username or len(request.username.strip()) == 0:
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Username cannot be empty")
    
    if len(request.username.strip()) < 3:
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Username must be at least 3 characters long")
    
    if len(request.username.strip()) > 50:
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Username cannot be longer than 50 characters")
    
    if not re.match(r'^[a-zA-Z0-9_-]+$', request.username.strip()):
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Username can only contain letters, numbers, underscores, and hyphens")
    
    # Validate email
    if not request.email or len(request.email.strip()) == 0:
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Email cannot be empty")
    
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(email_pattern, request.email.strip()):
        # TODO: Replace ValueError with UserValidationException
        raise ValueError("Invalid email format")
    
    return {"message": "User created successfully", "username": request.username, "email": request.email}

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": exc.errors()}
    )

@app.exception_handler(IntegrityError)
async def database_exception_handler(request: Request, exc: IntegrityError):
    logger.error(f"Database error: {str(exc)}")
    return JSONResponse(
        status_code=status.HTTP_409_CONFLICT,
        content={"error": "Database constraint violation"}
    )

def process_changeset_review(changeset_id: int):
    pass

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        
        # Basic security check - no parent directory access
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        
        return v.strip()
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Title cannot be empty')
        
        if len(v) > 200:
            raise ValueError('Title too long (max 200 characters)')
        
        return v.strip()
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Author cannot be empty')
        
        # Basic email or username validation
        if not re.match(r'^[a-zA-Z0-9._-]+(@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})?$', v):
            raise ValueError('Invalid author format')
        
        return v.strip()
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v):
        if not v or len(v) == 0:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:  # Limit to 20 files per changeset
            raise ValueError('Too many files (max 20)')
        
        return v

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_session)
):
    """Submit a new changeset for review with validation"""
    try:
        # Create changeset
        changeset = Changeset(
            title=request.title,
            description=request.description,
            author=request.author
        )
        db.add(changeset)
        db.flush()
        
        # Add files
        for file_req in request.files:
            changeset_file = ChangesetFile(
                changeset_id=changeset.id,
                file_path=file_req.file_path,
                diff_content=file_req.diff_content
            )
            db.add(changeset_file)
        
        db.commit()
        logger.info(f"Changeset {changeset.id} created by {request.author}")
        
        # Process review in background
        background_tasks.add_task(process_changeset_review, changeset.id)
        
        return {"message": "Changeset submitted", "changeset_id": changeset.id}
        
    except Exception as e:
        db.rollback()
        logger.error(f"Failed to create changeset: {str(e)}")
        raise CodeReviewException("Failed to create changeset", 500)

# Test the custom exception functionality
if __name__ == "__main__":
    print("Testing UserValidationException class...")
    
    # Test 1: Creating custom exception
    try:
        exc = UserValidationException("Test message", 400)
        print(f"✓ Custom exception created: {exc.message}, status_code: {exc.status_code}")
    except Exception as e:
        print(f"✗ Failed to create custom exception: {e}")
        exit(1)
    
    # Test 2: Testing validation logic
    test_cases = [
        {"username": "", "email": "test@example.com", "expected_error": "Username cannot be empty"},
        {"username": "ab", "email": "test@example.com", "expected_error": "Username must be at least 3 characters long"},
        {"username": "valid_user", "email": "", "expected_error": "Email cannot be empty"},
        {"username": "valid_user", "email": "invalid_email", "expected_error": "Invalid email format"},
    ]
    
    print("\nTesting validation logic...")
    for i, test_case in enumerate(test_cases, 1):
        try:
            request = UserRequest(username=test_case["username"], email=test_case["email"])
            create_user(request)
            print(f"✗ Test {i} failed: Expected error but validation passed")
        except UserValidationException as e:
            if e.message == test_case["expected_error"]:
                print(f"✓ Test {i} passed: {e.message}")
            else:
                print(f"✗ Test {i} failed: Expected '{test_case['expected_error']}', got '{e.message}'")
        except Exception as e:
            print(f"✗ Test {i} failed: Unexpected error type: {type(e).__name__}")
    
    # Test 3: Valid user creation
    try:
        request = UserRequest(username="valid_user", email="user@example.com")
        result = create_user(request)
        print(f"✓ Valid user creation test passed: {result['message']}")
    except Exception as e:
        print(f"✗ Valid user creation test failed: {e}")
    
    print("\n✓ All tests completed! Custom exception implementation is working correctly.")

```

## Implement Custom Exception Handler Function

Perfect! You've successfully built your custom exception class and learned how to organize error handling with domain-specific exceptions. Now it's time to complete the error-handling system by implementing the exception handler that will catch your custom exception and return proper JSON responses.

Currently, when your /api/users endpoint raises a UserValidationException, FastAPI doesn't know how to handle it properly, so users get generic 500 error responses instead of the clear, helpful messages you intended. Your objective is to create an exception handler that will fix this.

Here's what you need to do:

Create an async function that will handle UserValidationException instances.
Decorate it with @app.exception_handler(UserValidationException) to tell FastAPI to use it.
The function should accept two parameters: request: Request and exc: UserValidationException.
Log the validation error using the provided logger with a warning level.
Return a JSONResponse with the exception's status code and a JSON payload containing the error message and error type.
The JSON response should have this structure:

```JSON
Copy to clipboard
{
  "error": "the exception message",
  "type": "user_validation_error"
}
```

Once you implement this handler, users will receive clear, consistent error messages when they send invalid data to your API, completing your professional error-handling system!

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks, Depends
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import datetime
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

app = FastAPI()

class UserValidationException(Exception):
    """Custom exception for user validation errors"""
    def __init__(self, message: str, status_code: int = 400):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

# TODO: Add exception handler for UserValidationException that returns JSONResponse with error message and type

class UserRequest(BaseModel):
    username: str
    email: str

@app.post("/api/users", response_model=dict)
def create_user(request: UserRequest):
    """Create a new user with validation"""
    # Validate username
    if not request.username or len(request.username.strip()) == 0:
        raise UserValidationException("Username cannot be empty")
    
    if len(request.username.strip()) < 3:
        raise UserValidationException("Username must be at least 3 characters long")
    
    if len(request.username.strip()) > 50:
        raise UserValidationException("Username cannot be longer than 50 characters")
    
    if not re.match(r'^[a-zA-Z0-9_-]+$', request.username.strip()):
        raise UserValidationException("Username can only contain letters, numbers, underscores, and hyphens")
    
    # Validate email
    if not request.email or len(request.email.strip()) == 0:
        raise UserValidationException("Email cannot be empty")
    
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(email_pattern, request.email.strip()):
        raise UserValidationException("Invalid email format")
    
    return {"message": "User created successfully", "username": request.username, "email": request.email}

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": exc.errors()}
    )

@app.exception_handler(IntegrityError)
async def database_exception_handler(request: Request, exc: IntegrityError):
    logger.error(f"Database error: {str(exc)}")
    return JSONResponse(
        status_code=status.HTTP_409_CONFLICT,
        content={"error": "Database constraint violation"}
    )

# Database session dependency
def get_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Database models
class Changeset(Base):
    __tablename__ = "changesets"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200))
    description = Column(Text, nullable=True)
    author = Column(String(100))
    status = Column(String(20), default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

class ChangesetFile(Base):
    __tablename__ = "changeset_files"
    id = Column(Integer, primary_key=True, index=True)
    changeset_id = Column(Integer, ForeignKey("changesets.id"))
    file_path = Column(String(255))
    diff_content = Column(Text)

# Create database tables
Base.metadata.create_all(bind=engine)

def process_changeset_review(changeset_id: int):
    """Background task to process changeset review"""
    db = SessionLocal()
    try:
        changeset = db.query(Changeset).filter_by(id=changeset_id).first()
        if changeset:
            changeset.status = 'reviewed'
            db.commit()
            logger.info(f"Changeset {changeset_id} reviewed")
    finally:
        db.close()

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        
        # Basic security check - no parent directory access
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        
        return v.strip()
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Title cannot be empty')
        
        if len(v) > 200:
            raise ValueError('Title too long (max 200 characters)')
        
        return v.strip()
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Author cannot be empty')
        
        # Basic email or username validation
        if not re.match(r'^[a-zA-Z0-9._-]+(@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})?$', v):
            raise ValueError('Invalid author format')
        
        return v.strip()
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v):
        if not v or len(v) == 0:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:  # Limit to 20 files per changeset
            raise ValueError('Too many files (max 20)')
        
        return v

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_session)
):
    """Submit a new changeset for review with validation"""
    try:
        # Create changeset
        changeset = Changeset(
            title=request.title,
            description=request.description,
            author=request.author
        )
        db.add(changeset)
        db.flush()
        
        # Add files
        for file_req in request.files:
            changeset_file = ChangesetFile(
                changeset_id=changeset.id,
                file_path=file_req.file_path,
                diff_content=file_req.diff_content
            )
            db.add(changeset_file)
        
        db.commit()
        logger.info(f"Changeset {changeset.id} created by {request.author}")
        
        # Process review in background
        background_tasks.add_task(process_changeset_review, changeset.id)
        
        return {"message": "Changeset submitted", "changeset_id": changeset.id}
        
    except Exception as e:
        db.rollback()
        logger.error(f"Failed to create changeset: {str(e)}")
        raise CodeReviewException("Failed to create changeset", 500)

if __name__ == "__main__":
    import uvicorn
    from fastapi.testclient import TestClient
    
    # Create test client
    client = TestClient(app)
    
    print("Testing UserValidationException handler...")
    
    # Test case 1: Empty username
    print("\n1. Testing empty username:")
    response = client.post("/api/users", json={"username": "", "email": "test@example.com"})
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 2: Invalid email
    print("\n2. Testing invalid email:")
    response = client.post("/api/users", json={"username": "testuser", "email": "invalid-email"})
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 3: Valid data
    print("\n3. Testing valid data:")
    response = client.post("/api/users", json={"username": "validuser", "email": "valid@example.com"})
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    print("\n✅ Exception handler testing completed!")

```

Kerja yang luar biasa\! Anda telah membuat *custom exception handler* `UserValidationException` di FastAPI. Ini akan memastikan *endpoint* `/api/users` Anda mengembalikan respons JSON yang jelas dan konsisten alih-alih kesalahan server generik.

Berikut adalah kode yang telah diselesaikan dengan *exception handler* yang diimplementasikan:

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks, Depends
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import datetime
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

app = FastAPI()

class UserValidationException(Exception):
    """Custom exception for user validation errors"""
    def __init__(self, message: str, status_code: int = 400):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

# ✅ TODO: Add exception handler for UserValidationException that returns JSONResponse with error message and type
@app.exception_handler(UserValidationException)
async def user_validation_exception_handler(request: Request, exc: UserValidationException):
    """
    Handles UserValidationException, logging the error and returning a structured JSON response.
    """
    # Log the validation error
    logger.warning(f"User validation failed: {exc.message} on request {request.url} (Status: {exc.status_code})")
    
    # Return a JSONResponse with the custom status code and structured content
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "user_validation_error"}
    )

class UserRequest(BaseModel):
    username: str
    email: str

@app.post("/api/users", response_model=dict)
def create_user(request: UserRequest):
    """Create a new user with validation"""
    # Validate username
    if not request.username or len(request.username.strip()) == 0:
        raise UserValidationException("Username cannot be empty")
    
    if len(request.username.strip()) < 3:
        raise UserValidationException("Username must be at least 3 characters long")
    
    if len(request.username.strip()) > 50:
        raise UserValidationException("Username cannot be longer than 50 characters")
    
    if not re.match(r'^[a-zA-Z0-9_-]+$', request.username.strip()):
        raise UserValidationException("Username can only contain letters, numbers, underscores, and hyphens")
    
    # Validate email
    if not request.email or len(request.email.strip()) == 0:
        raise UserValidationException("Email cannot be empty")
    
    email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if not re.match(email_pattern, request.email.strip()):
        raise UserValidationException("Invalid email format")
    
    return {"message": "User created successfully", "username": request.username, "email": request.email}

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": exc.errors()}
    )

@app.exception_handler(IntegrityError)
async def database_exception_handler(request: Request, exc: IntegrityError):
    logger.error(f"Database error: {str(exc)}")
    return JSONResponse(
        status_code=status.HTTP_409_CONFLICT,
        content={"error": "Database constraint violation"}
    )

# Database session dependency
def get_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# Database models
class Changeset(Base):
    __tablename__ = "changesets"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200))
    description = Column(Text, nullable=True)
    author = Column(String(100))
    status = Column(String(20), default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

class ChangesetFile(Base):
    __tablename__ = "changeset_files"
    id = Column(Integer, primary_key=True, index=True)
    changeset_id = Column(Integer, ForeignKey("changesets.id"))
    file_path = Column(String(255))
    diff_content = Column(Text)

# Create database tables
Base.metadata.create_all(bind=engine)

def process_changeset_review(changeset_id: int):
    """Background task to process changeset review"""
    db = SessionLocal()
    try:
        changeset = db.query(Changeset).filter_by(id=changeset_id).first()
        if changeset:
            changeset.status = 'reviewed'
            db.commit()
            logger.info(f"Changeset {changeset_id} reviewed")
    finally:
        db.close()

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        
        # Basic security check - no parent directory access
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        
        return v.strip()
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Title cannot be empty')
        
        if len(v) > 200:
            raise ValueError('Title too long (max 200 characters)')
        
        return v.strip()
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Author cannot be empty')
        
        # Basic email or username validation
        if not re.match(r'^[a-zA-Z0-9._-]+(@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})?$', v):
            raise ValueError('Invalid author format')
        
        return v.strip()
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v):
        if not v or len(v) == 0:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:  # Limit to 20 files per changeset
            raise ValueError('Too many files (max 20)')
        
        return v

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_session)
):
    """Submit a new changeset for review with validation"""
    try:
        # Create changeset
        changeset = Changeset(
            title=request.title,
            description=request.description,
            author=request.author
        )
        db.add(changeset)
        db.flush()
        
        # Add files
        for file_req in request.files:
            changeset_file = ChangesetFile(
                changeset_id=changeset.id,
                file_path=file_req.file_path,
                diff_content=file_req.diff_content
            )
            db.add(changeset_file)
        
        db.commit()
        logger.info(f"Changeset {changeset.id} created by {request.author}")
        
        # Process review in background
        background_tasks.add_task(process_changeset_review, changeset.id)
        
        return {"message": "Changeset submitted", "changeset_id": changeset.id}
        
    except Exception as e:
        db.rollback()
        logger.error(f"Failed to create changeset: {str(e)}")
        raise CodeReviewException("Failed to create changeset", 500)

if __name__ == "__main__":
    import uvicorn
    from fastapi.testclient import TestClient
    
    # Create test client
    client = TestClient(app)
    
    print("Testing UserValidationException handler...")
    
    # Test case 1: Empty username -> Should return 400 Bad Request
    print("\n1. Testing empty username:")
    response = client.post("/api/users", json={"username": "", "email": "test@example.com"})
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 2: Invalid email -> Should return 400 Bad Request
    print("\n2. Testing invalid email:")
    response = client.post("/api/users", json={"username": "testuser", "email": "invalid-email"})
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 3: Valid data -> Should return 200 OK
    print("\n3. Testing valid data:")
    response = client.post("/api/users", json={"username": "validuser", "email": "valid@example.com"})
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    print("\n✅ Exception handler testing completed! The API now provides clear, domain-specific error responses.")
```

-----

### Detail Implementasi

Anda telah menambahkan fungsi *handler* yang diperlukan:

```python
@app.exception_handler(UserValidationException)
async def user_validation_exception_handler(request: Request, exc: UserValidationException):
    # Log the validation error
    logger.warning(f"User validation failed: {exc.message} on request {request.url} (Status: {exc.status_code})")
    
    # Return a JSONResponse with the custom status code and structured content
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "user_validation_error"}
    )
```

1.  **Dekorator:** `@app.exception_handler(UserValidationException)` memberi tahu FastAPI bahwa setiap kali `UserValidationException` di-`raise`, fungsi ini harus dijalankan untuk menangani responsnya.
2.  **Parameter:** Fungsi ini menerima `Request` (untuk konteks) dan objek `UserValidationException` (`exc`) yang dilempar.
3.  **Logging:** `logger.warning(...)` mencatat kesalahan untuk pemantauan backend tanpa mengekspos detail internal kepada pengguna.
4.  **Respons:** `JSONResponse` dibuat menggunakan `exc.status_code` dan *payload* JSON yang diminta: `{"error": exc.message, "type": "user_validation_error"}`.

Ini adalah praktik terbaik di FastAPI untuk memisahkan **logika bisnis** (validasi, yang melempar pengecualian) dari **logika presentasi** (*handler*, yang mengubah pengecualian menjadi respons HTTP).

## Add Database Transaction Error Handling

Fantastic work building custom exceptions and exception handlers! Now you'll learn one of the most critical skills in professional API development: database transaction management and error handling.

You have a working changeset submission endpoint that saves records to a database, but there's a serious problem. When database operations fail (like trying to create a changeset with invalid data or when database constraints are violated), the endpoint doesn't handle these errors properly, leaving your database in an inconsistent state.

Your task is to add proper database error handling to the existing submit_changeset endpoint to make it reliable and safe:

Wrap all database operations in a try-catch block to handle potential errors.
Catch IntegrityError exceptions specifically (these occur when database constraints are violated).
When an IntegrityError occurs, call db.rollback() to undo any partial changes, then raise a CodeReviewException.
For successful operations, call db.commit() to save changes permanently.
Handle any other unexpected database errors with proper rollback and error reporting.
This pattern of try-catch-rollback-commit is essential for building reliable database-driven APIs and ensures your database stays consistent even when things go wrong!

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks, Depends
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session, sessionmaker, declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re
from datetime import datetime

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Database models
class Changeset(Base):
    __tablename__ = "changesets"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200))
    description = Column(Text, nullable=True)
    author = Column(String(100))
    status = Column(String(20), default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

class ChangesetFile(Base):
    __tablename__ = "changeset_files"
    id = Column(Integer, primary_key=True, index=True)
    changeset_id = Column(Integer, ForeignKey("changesets.id"))
    file_path = Column(String(255))
    diff_content = Column(Text)

# Create database tables
Base.metadata.create_all(bind=engine)

def get_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        
        # Basic security check - no parent directory access
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        
        return v.strip()
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Title cannot be empty')
        
        if len(v) > 200:
            raise ValueError('Title too long (max 200 characters)')
        
        return v.strip()
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Author cannot be empty')
        
        # Basic email or username validation
        if not re.match(r'^[a-zA-Z0-9._-]+(@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})?$', v):
            raise ValueError('Invalid author format')
        
        return v.strip()
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v):
        if not v or len(v) == 0:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:  # Limit to 20 files per changeset
            raise ValueError('Too many files (max 20)')
        
        return v

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    # Convert errors to JSON-serializable format
    serializable_errors = []
    for error in exc.errors():
        serializable_error = {
            "type": error.get("type"),
            "loc": error.get("loc"),
            "msg": error.get("msg"),
            "input": str(error.get("input"))
        }
        serializable_errors.append(serializable_error)
    
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": serializable_errors}
    )

@app.exception_handler(IntegrityError)
async def database_exception_handler(request: Request, exc: IntegrityError):
    logger.error(f"Database error: {str(exc)}")
    return JSONResponse(
        status_code=status.HTTP_409_CONFLICT,
        content={"error": "Database constraint violation"}
    )

def process_changeset_review(changeset_id: int):
    pass

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_session)
):
    """Submit a new changeset for review with proper database error handling"""
    # TODO: Add try-catch block to handle database errors
    # Create changeset
    changeset = Changeset(
        title=request.title,
        description=request.description,
        author=request.author
    )
    db.add(changeset)
    db.flush()  # This might raise IntegrityError
    
    # Add files
    for file_req in request.files:
        changeset_file = ChangesetFile(
            changeset_id=changeset.id,
            file_path=file_req.file_path,
            diff_content=file_req.diff_content
        )
        db.add(changeset_file)
    
    # TODO: Add db.commit() to save successful operations
    
    logger.info(f"Changeset {changeset.id} created by {request.author}")
    
    # Process review in background
    background_tasks.add_task(process_changeset_review, changeset.id)
    
    return {"message": "Changeset submitted", "changeset_id": changeset.id}
    
    # TODO: Add except IntegrityError block - call db.rollback() and raise CodeReviewException with message "Database constraint violation - duplicate data or invalid foreign key" and status 409
    
    # TODO: Add except Exception block - call db.rollback() and raise CodeReviewException with message "Failed to create changeset" and status 500

if __name__ == "__main__":
    from fastapi.testclient import TestClient
    
    # Create test client
    client = TestClient(app)
    
    print("Testing database error handling...")
    
    # Test case 1: Valid changeset creation
    print("\n1. Testing valid changeset creation:")
    response = client.post("/api/changesets", json={
        "title": "Add new feature",
        "author": "developer@example.com",
        "files": [
            {
                "file_path": "src/main.py",
                "diff_content": "@@ -1,5 +1,7 @@\n+def new_function():\n+    return True"
            }
        ]
    })
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 2: Invalid input (should trigger validation error)
    print("\n2. Testing invalid input:")
    response = client.post("/api/changesets", json={
        "title": "",  # Empty title should fail validation
        "author": "developer@example.com",
        "files": []
    })
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 3: Invalid file path
    print("\n3. Testing invalid file path:")
    response = client.post("/api/changesets", json={
        "title": "Test changeset",
        "author": "developer@example.com",
        "files": [
            {
                "file_path": "../../../etc/passwd",  # Should fail validation
                "diff_content": "malicious content"
            }
        ]
    })
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    print("\n✅ Database error handling testing completed!")

```

Ini adalah langkah penting\! Mengelola transaksi basis data dengan benar adalah kunci untuk API yang **andal dan konsisten**.

Saya telah mengimplementasikan blok `try-except-commit-rollback` di *endpoint* `submit_changeset` sesuai dengan praktik terbaik untuk menangani transaksi SQLAlchemy.

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks, Depends
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session, sessionmaker, declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re
from datetime import datetime

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Database models
class Changeset(Base):
    __tablename__ = "changesets"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200))
    description = Column(Text, nullable=True)
    author = Column(String(100))
    status = Column(String(20), default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

class ChangesetFile(Base):
    __tablename__ = "changeset_files"
    id = Column(Integer, primary_key=True, index=True)
    changeset_id = Column(Integer, ForeignKey("changesets.id"))
    file_path = Column(String(255))
    diff_content = Column(Text)

# Create database tables
Base.metadata.create_all(bind=engine)

def get_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('File path cannot be empty')
        
        # Basic security check - no parent directory access
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path')
        
        return v.strip()
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Diff content cannot be empty')
        
        if len(v) > 50000:  # 50KB limit
            raise ValueError('Diff content too large')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Title cannot be empty')
        
        if len(v) > 200:
            raise ValueError('Title too long (max 200 characters)')
        
        return v.strip()
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v):
        if not v or len(v.strip()) == 0:
            raise ValueError('Author cannot be empty')
        
        # Basic email or username validation
        if not re.match(r'^[a-zA-Z0-9._-]+(@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})?$', v):
            raise ValueError('Invalid author format')
        
        return v.strip()
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v):
        if not v or len(v) == 0:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:  # Limit to 20 files per changeset
            raise ValueError('Too many files (max 20)')
        
        return v

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    # Convert errors to JSON-serializable format
    serializable_errors = []
    for error in exc.errors():
        serializable_error = {
            "type": error.get("type"),
            "loc": error.get("loc"),
            "msg": error.get("msg"),
            "input": str(error.get("input"))
        }
        serializable_errors.append(serializable_error)
    
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": serializable_errors}
    )

@app.exception_handler(IntegrityError)
async def database_exception_handler(request: Request, exc: IntegrityError):
    logger.error(f"Database error: {str(exc)}")
    return JSONResponse(
        status_code=status.HTTP_409_CONFLICT,
        content={"error": "Database constraint violation"}
    )

def process_changeset_review(changeset_id: int):
    pass

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_session)
):
    """Submit a new changeset for review with proper database error handling"""
    # ✅ Add try-catch block to handle database errors
    try:
        # Create changeset
        changeset = Changeset(
            title=request.title,
            description=request.description,
            author=request.author
        )
        db.add(changeset)
        db.flush()  # Ensures the changeset gets an ID and can expose IntegrityError
        
        # Add files
        for file_req in request.files:
            changeset_file = ChangesetFile(
                changeset_id=changeset.id,
                file_path=file_req.file_path,
                diff_content=file_req.diff_content
            )
            db.add(changeset_file)
        
        # ✅ Add db.commit() to save successful operations
        db.commit()
        
        logger.info(f"Changeset {changeset.id} created by {request.author}")
        
        # Process review in background
        background_tasks.add_task(process_changeset_review, changeset.id)
        
        return {"message": "Changeset submitted", "changeset_id": changeset.id}
        
    # ✅ Catch IntegrityError, call db.rollback(), and raise CodeReviewException (409)
    except IntegrityError as e:
        db.rollback()
        logger.error(f"Integrity Error during changeset creation: {str(e)}")
        raise CodeReviewException(
            "Database constraint violation - duplicate data or invalid foreign key", 
            status_code=status.HTTP_409_CONFLICT # 409 Conflict is appropriate for integrity errors
        )
    
    # ✅ Catch any other unexpected database errors, call db.rollback(), and raise CodeReviewException (500)
    except Exception as e:
        db.rollback()
        logger.error(f"Unexpected error during changeset creation: {str(e)}")
        # Re-raise if it's already a CodeReviewException, otherwise raise a new one
        if isinstance(e, CodeReviewException):
            raise e
        
        raise CodeReviewException("Failed to create changeset due to an unexpected database error", status_code=status.HTTP_500_INTERNAL_SERVER_ERROR)

if __name__ == "__main__":
    from fastapi.testclient import TestClient
    
    # Create test client
    client = TestClient(app)
    
    print("Testing database error handling...")
    
    # Test case 1: Valid changeset creation
    print("\n1. Testing valid changeset creation:")
    response = client.post("/api/changesets", json={
        "title": "Add new feature",
        "author": "developer@example.com",
        "files": [
            {
                "file_path": "src/main.py",
                "diff_content": "@@ -1,5 +1,7 @@\n+def new_function():\n+    return True"
            }
        ]
    })
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 2: Invalid input (should trigger Pydantic/Validator error, resulting in 422)
    print("\n2. Testing invalid input (Pydantic/Validator):")
    response = client.post("/api/changesets", json={
        "title": "",  # Empty title should fail validation
        "author": "developer@example.com",
        "files": []
    })
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 3: Invalid file path (Pydantic/Validator error, resulting in 422)
    print("\n3. Testing invalid file path (Pydantic/Validator):")
    response = client.post("/api/changesets", json={
        "title": "Test changeset",
        "author": "developer@example.com",
        "files": [
            {
                "file_path": "../../../etc/passwd",  # Should fail validation
                "diff_content": "malicious content"
            }
        ]
    })
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    print("\n✅ Database error handling testing completed! The endpoint is now transactional.")
```

### Penjelasan Implementasi

1.  **Blok `try`:** Semua operasi basis data (`db.add`, `db.flush`, `db.add` file, `db.commit`) dibungkus.
2.  **`db.flush()`:** Ini opsional tetapi sering digunakan. Ini mengirimkan pernyataan SQL ke basis data untuk mendapatkan ID primer yang baru dibuat (`changeset.id`) dan juga dapat memicu `IntegrityError` lebih awal sebelum `db.commit()`.
3.  **`db.commit()`:** Hanya dipanggil di akhir blok `try`, memastikan bahwa semua operasi basis data (membuat `Changeset` dan semua `ChangesetFile`) berhasil dilakukan sebagai satu **transaksi atomik** sebelum disimpan secara permanen.
4.  **`except IntegrityError`:**
      * **`db.rollback()`:** Ini adalah langkah paling penting. Jika ada pelanggaran batasan (misalnya, nilai terlalu panjang atau batasan `UNIQUE` dilanggar), basis data berada dalam keadaan tidak valid. `rollback()` membatalkan semua perubahan yang tertunda dalam transaksi ini, mengembalikan status sesi menjadi bersih.
      * `raise CodeReviewException(..., status_code=409)`: Meneruskan kesalahan ke *handler* pengecualian FastAPI dengan kode status **$409$ Conflict**, yang tepat untuk pelanggaran batasan.
5.  **`except Exception`:**
      * **`db.rollback()`:** Menangkap kesalahan basis data lain yang tidak terduga, juga melakukan *rollback* untuk keamanan.
      * Menaikkan `CodeReviewException` dengan kode status **$500$ Internal Server Error** untuk menangani kegagalan sistem yang tidak terduga.

## Build Production Ready Changeset Endpoint

Outstanding work mastering database transaction management! Now it's time to bring together everything you've learned in this unit by building a comprehensive, production-ready changeset submission endpoint.

You have a changeset submission endpoint that allows developers to submit code changes for review, but it's missing the critical validation and error handling that make APIs safe and reliable. Your objective is to transform this basic endpoint into a robust, secure system that can handle real-world usage.

Here's what you need to implement:

Add comprehensive field validation:

Validate the title field for empty values and enforce a 200-character limit.
Validate the author field using proper email format validation.
Secure the file_path field against directory traversal attacks.
Enforce a 50 KB size limit on diff_content to prevent abuse.
Implement proper database transaction management:

Wrap all database operations in try-catch blocks.
Add proper commit and rollback logic to maintain data consistency.
Handle constraint violations and unexpected database errors appropriately.
This exercise combines all the concepts you've mastered — Pydantic field validators, custom exceptions, exception handlers, and database transaction safety — into a single endpoint that's ready for production use!


```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks, Depends
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import datetime
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Database models
class Changeset(Base):
    __tablename__ = "changesets"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200))
    description = Column(Text, nullable=True)
    author = Column(String(100))
    status = Column(String(20), default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

class ChangesetFile(Base):
    __tablename__ = "changeset_files"
    id = Column(Integer, primary_key=True, index=True)
    changeset_id = Column(Integer, ForeignKey("changesets.id"))
    file_path = Column(String(255))
    diff_content = Column(Text)

# Create database tables
Base.metadata.create_all(bind=engine)

def get_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    # TODO: Add field validator for file_path that checks for empty values and prevents directory traversal (no '..' or '/' prefixes)
    
    # TODO: Add field validator for diff_content that checks for empty values and enforces 50KB size limit

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    # TODO: Add field validator for title that checks for empty values and enforces max 200 characters
    
    # TODO: Add field validator for author that validates email format using regex pattern

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message}")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    logger.warning(f"Validation error: {exc.errors()}")
    
    # Clean up errors to make them JSON serializable
    cleaned_errors = []
    for error in exc.errors():
        cleaned_error = {
            "type": error.get("type"),
            "loc": error.get("loc"),
            "msg": error.get("msg"),
            "input": str(error.get("input", ""))[:100] + "..." if len(str(error.get("input", ""))) > 100 else str(error.get("input", ""))
        }
        cleaned_errors.append(cleaned_error)
    
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": cleaned_errors}
    )

@app.exception_handler(IntegrityError)
async def database_exception_handler(request: Request, exc: IntegrityError):
    logger.error(f"Database error: {str(exc)}")
    return JSONResponse(
        status_code=status.HTTP_409_CONFLICT,
        content={"error": "Database constraint violation"}
    )

def process_changeset_review(changeset_id: int):
    pass

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_session)
):
    """Submit a new changeset for review with validation"""
    # TODO: Add try block to wrap all database operations
    # Create changeset
    changeset = Changeset(
        title=request.title,
        description=request.description,
        author=request.author
    )
    db.add(changeset)
    db.flush()
    
    # Add files
    for file_req in request.files:
        changeset_file = ChangesetFile(
            changeset_id=changeset.id,
            file_path=file_req.file_path,
            diff_content=file_req.diff_content
        )
        db.add(changeset_file)
    
    # TODO: Add db.commit() to save successful operations
    logger.info(f"Changeset {changeset.id} created by {request.author}")
    
    # Process review in background
    background_tasks.add_task(process_changeset_review, changeset.id)
    
    return {"message": "Changeset submitted", "changeset_id": changeset.id}
    
    # TODO: Add except IntegrityError block - call db.rollback() and raise CodeReviewException with message "Database constraint violation" and status 409
    
    # TODO: Add except Exception block - call db.rollback() and raise CodeReviewException with message "Failed to create changeset" and status 500

if __name__ == "__main__":
    from fastapi.testclient import TestClient
    
    # Create test client
    client = TestClient(app)
    
    print("Testing comprehensive changeset validation and error handling...")
    
    # Test case 1: Valid changeset submission
    print("\n1. Testing valid changeset:")
    valid_changeset = {
        "title": "Fix authentication bug",
        "description": "Updated login validation logic",
        "author": "developer@example.com",
        "files": [
            {
                "file_path": "src/auth.py",
                "diff_content": "- old_code\n+ new_code"
            }
        ]
    }
    response = client.post("/api/changesets", json=valid_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 2: Invalid title (too long)
    print("\n2. Testing title too long:")
    long_title_changeset = {
        "title": "A" * 250,  # Exceeds 200 character limit
        "author": "developer@example.com",
        "files": [{"file_path": "test.py", "diff_content": "test"}]
    }
    response = client.post("/api/changesets", json=long_title_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 3: Invalid email format
    print("\n3. Testing invalid email:")
    invalid_email_changeset = {
        "title": "Test changeset",
        "author": "not-an-email",
        "files": [{"file_path": "test.py", "diff_content": "test"}]
    }
    response = client.post("/api/changesets", json=invalid_email_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 4: Invalid file path (directory traversal)
    print("\n4. Testing directory traversal attack:")
    traversal_changeset = {
        "title": "Test changeset",
        "author": "developer@example.com",
        "files": [{"file_path": "../../../etc/passwd", "diff_content": "malicious"}]
    }
    response = client.post("/api/changesets", json=traversal_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 5: Content too large
    print("\n5. Testing content size limit:")
    large_content_changeset = {
        "title": "Test changeset",
        "author": "developer@example.com",
        "files": [{"file_path": "test.py", "diff_content": "x" * 60000}]  # Exceeds 50KB
    }
    response = client.post("/api/changesets", json=large_content_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    print("\n✅ Comprehensive validation and error handling testing completed!")

```

Sangat baik\! Membangun *endpoint* dengan validasi yang ketat dan manajemen transaksi basis data yang kuat adalah ciri dari API siap produksi.

Berikut adalah kode yang telah diselesaikan. Saya telah mengintegrasikan semua **Pydantic field validators** yang diperlukan untuk keamanan dan batasan data, serta menambahkan **logika `try-except-commit-rollback`** yang lengkap ke *endpoint* `submit_changeset`.

```python
from fastapi import FastAPI, HTTPException, Request, status, BackgroundTasks, Depends
from fastapi.responses import JSONResponse
from fastapi.exceptions import RequestValidationError
from sqlalchemy.exc import IntegrityError
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from datetime import datetime
import logging
from typing import Union, List, Optional
from pydantic import BaseModel, field_validator
import re
import math # Import math for size calculation

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI()

# Database setup
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
Base = declarative_base()

# Database models
class Changeset(Base):
    __tablename__ = "changesets"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String(200))
    description = Column(Text, nullable=True)
    author = Column(String(100))
    status = Column(String(20), default="pending")
    created_at = Column(DateTime, default=datetime.utcnow)

class ChangesetFile(Base):
    __tablename__ = "changeset_files"
    id = Column(Integer, primary_key=True, index=True)
    changeset_id = Column(Integer, ForeignKey("changesets.id"))
    file_path = Column(String(255))
    diff_content = Column(Text)

# Create database tables
Base.metadata.create_all(bind=engine)

def get_session():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

# --- Model & Validation Implementation ---

# Constants for validation
MAX_TITLE_LENGTH = 200
MAX_DIFF_SIZE_BYTES = 50 * 1024 # 50 KB

class FileChangeRequest(BaseModel):
    file_path: str
    diff_content: str
    
    @field_validator('file_path')
    @classmethod
    def validate_file_path(cls, v: str):
        v = v.strip()
        if not v:
            raise ValueError('File path cannot be empty')
        
        # ✅ Secure against directory traversal (no '..' or leading '/')
        if '..' in v or v.startswith('/'):
            raise ValueError('Invalid file path: directory traversal not allowed')
        
        return v
    
    @field_validator('diff_content')
    @classmethod
    def validate_diff_content(cls, v: str):
        v = v.strip()
        if not v:
            raise ValueError('Diff content cannot be empty')
        
        # ✅ Enforce 50KB size limit
        if len(v.encode('utf-8')) > MAX_DIFF_SIZE_BYTES:
            size_kb = len(v.encode('utf-8')) / 1024
            max_kb = MAX_DIFF_SIZE_BYTES / 1024
            raise ValueError(f'Diff content too large ({math.ceil(size_kb)} KB). Max size is {max_kb} KB')
        
        return v

class ChangesetRequest(BaseModel):
    title: str
    description: Optional[str] = None
    author: str
    files: List[FileChangeRequest]
    
    @field_validator('title')
    @classmethod
    def validate_title(cls, v: str):
        v = v.strip()
        if not v:
            raise ValueError('Title cannot be empty')
        
        # ✅ Enforce 200-character limit
        if len(v) > MAX_TITLE_LENGTH:
            raise ValueError(f'Title too long (max {MAX_TITLE_LENGTH} characters)')
        
        return v
    
    @field_validator('author')
    @classmethod
    def validate_author(cls, v: str):
        v = v.strip()
        if not v:
            raise ValueError('Author cannot be empty')
        
        # ✅ Validate email format
        email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not re.match(email_pattern, v):
            raise ValueError('Invalid author format. Must be a valid email address.')
        
        return v
    
    @field_validator('files')
    @classmethod
    def validate_files(cls, v: List[FileChangeRequest]):
        if not v:
            raise ValueError('At least one file must be provided')
        
        if len(v) > 20:
            raise ValueError('Too many files (max 20)')
        
        return v

# --- Custom Exceptions and Handlers ---

class CodeReviewException(Exception):
    """Custom exception for code review operations"""
    def __init__(self, message: str, status_code: int = 500):
        self.message = message
        self.status_code = status_code
        super().__init__(self.message)

@app.exception_handler(CodeReviewException)
async def code_review_exception_handler(request: Request, exc: CodeReviewException):
    logger.error(f"Code review error: {exc.message} (Status: {exc.status_code})")
    return JSONResponse(
        status_code=exc.status_code,
        content={"error": exc.message, "type": "code_review_error"}
    )

@app.exception_handler(RequestValidationError)
async def validation_exception_handler(request: Request, exc: RequestValidationError):
    # ... (existing handler logic for Pydantic errors)
    logger.warning(f"Validation error: {exc.errors()}")
    cleaned_errors = []
    for error in exc.errors():
        cleaned_error = {
            "type": error.get("type"),
            "loc": error.get("loc"),
            "msg": error.get("msg"),
            "input": str(error.get("input", ""))[:100] + "..." if len(str(error.get("input", ""))) > 100 else str(error.get("input", ""))
        }
        cleaned_errors.append(cleaned_error)
    
    return JSONResponse(
        status_code=status.HTTP_422_UNPROCESSABLE_ENTITY,
        content={"error": "Invalid input data", "details": cleaned_errors}
    )

def process_changeset_review(changeset_id: int):
    pass

# --- Production Ready Endpoint ---

@app.post("/api/changesets", response_model=dict)
def submit_changeset(
    request: ChangesetRequest,
    background_tasks: BackgroundTasks,
    db: Session = Depends(get_session)
):
    """Submit a new changeset for review with full validation and transactional safety."""
    
    # ✅ Add try block to wrap all database operations
    try:
        # 1. Create Changeset record
        changeset = Changeset(
            title=request.title,
            description=request.description,
            author=request.author
        )
        db.add(changeset)
        db.flush() # Flush to get the changeset ID and check for immediate DB integrity errors
        
        # 2. Add files
        for file_req in request.files:
            changeset_file = ChangesetFile(
                changeset_id=changeset.id,
                file_path=file_req.file_path,
                diff_content=file_req.diff_content
            )
            db.add(changeset_file)
        
        # 3. ✅ Add db.commit() to save successful operations
        db.commit()
        
        logger.info(f"Changeset {changeset.id} created by {request.author}")
        
        # 4. Process review in background
        background_tasks.add_task(process_changeset_review, changeset.id)
        
        return {"message": "Changeset submitted", "changeset_id": changeset.id}
        
    # 5. ✅ Handle IntegrityError (Constraint Violation)
    except IntegrityError as e:
        db.rollback()
        logger.error(f"DB Integrity Error during changeset creation: {str(e)}")
        # Raise a custom exception with 409 Conflict
        raise CodeReviewException(
            "Database constraint violation (e.g., duplicate data or invalid relationship)", 
            status_code=status.HTTP_409_CONFLICT
        )
    
    # 6. ✅ Handle other unexpected database errors
    except Exception as e:
        db.rollback()
        logger.error(f"Unexpected error during changeset creation: {str(e)}")
        # Raise a custom exception with 500 Internal Server Error
        raise CodeReviewException(
            "Failed to create changeset due to an unexpected server error.", 
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR
        )

# --- Test Execution ---
if __name__ == "__main__":
    from fastapi.testclient import TestClient
    
    # Create test client
    client = TestClient(app)
    
    print("Testing comprehensive changeset validation and error handling...")
    
    # Test case 1: Valid changeset submission
    print("\n1. Testing valid changeset:")
    valid_changeset = {
        "title": "Fix authentication bug",
        "description": "Updated login validation logic",
        "author": "developer@example.com",
        "files": [
            {
                "file_path": "src/auth.py",
                "diff_content": "- old_code\n+ new_code"
            }
        ]
    }
    response = client.post("/api/changesets", json=valid_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 2: Invalid title (too long) -> Expect 422
    print("\n2. Testing title too long:")
    long_title_changeset = {
        "title": "A" * 250,
        "author": "developer@example.com",
        "files": [{"file_path": "test.py", "diff_content": "test"}]
    }
    response = client.post("/api/changesets", json=long_title_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 3: Invalid email format -> Expect 422
    print("\n3. Testing invalid email:")
    invalid_email_changeset = {
        "title": "Test changeset",
        "author": "not-an-email",
        "files": [{"file_path": "test.py", "diff_content": "test"}]
    }
    response = client.post("/api/changesets", json=invalid_email_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 4: Invalid file path (directory traversal) -> Expect 422
    print("\n4. Testing directory traversal attack:")
    traversal_changeset = {
        "title": "Test changeset",
        "author": "developer@example.com",
        "files": [{"file_path": "../../../etc/passwd", "diff_content": "malicious"}]
    }
    response = client.post("/api/changesets", json=traversal_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 5: Content too large -> Expect 422
    print("\n5. Testing content size limit:")
    # Calculate size of 60000 characters in bytes (assuming 1 byte per char for simplicity)
    large_content_changeset = {
        "title": "Test changeset",
        "author": "developer@example.com",
        "files": [{"file_path": "test.py", "diff_content": "x" * 60000}]
    }
    response = client.post("/api/changesets", json=large_content_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    # Test case 6: Testing second successful submission (to ensure database is still clean)
    print("\n6. Testing second successful submission:")
    response = client.post("/api/changesets", json=valid_changeset)
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
    
    print("\n✅ Comprehensive validation and error handling testing completed! This endpoint is production-ready.")
```